# Colab setup


In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
True


In [0]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image

In [0]:
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import os
from PIL import Image

In [0]:
!wget https://cdn.skillenza.com/files/5ee0a2e1-bf6d-4173-8244-e75fa2d7bbe2/training.5k.zip
!unzip training.5k.zip  

In [0]:
!wget https://cdn.skillenza.com/files/7da538a3-4db6-46ea-a4f8-87a21368e5f5/testing.40k.zip
!unzip testing.40k.zip

In [0]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


# Hyperparameters

In [0]:
NUM_EPOCHS = 5
LR = 0.8*(1e-4)
BATCH_SIZE = 4
GAMMA = 0.95
curr_lr = LR

In [0]:
tfms = transforms.Compose([transforms.Resize((224,224)),transforms.transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [0]:
categories = pd.read_csv("/content/training/solution.csv")["category"]
def getitem(idx):
    img_name = os.path.join("/content/training/training/", str(idx+1) + ".png")
    image = Image.open(img_name)
    if tfms:
        image = tfms(image)
    category = categories[idx]
    sample = {'image': image, 'category': category}

    return sample

In [0]:
a = np.arange(0,5000)
np.random.shuffle(a)
train_indexes = a[:4901]
test_indexes = a[4901:5001]

# Model

In [0]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [0]:
model = models.resnet152(pretrained=True)

model = nn.Sequential(*list(model.children())[:-2],
                     nn.Conv2d(2048, 6, 3, 1),
                     nn.AdaptiveAvgPool2d(1), Flatten(),
                     nn.LogSoftmax())

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:04<00:00, 56694876.76it/s]


In [0]:
#print(model.children)

count = 0
for param in model.parameters():
    param.requires_grad = False
    count += 1
    if count >= 8:
        break

In [0]:
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Training

In [0]:
train_losses = []
train_accuracies = []
validation_accuracies = []

In [0]:
def validation_accuracy(model):
  """
  Performs inference on the Validation set and returns the reported accuracy
  """
  correct = 0
  
  with torch.no_grad():
    for idx in range(0,99):
      data = getitem(test_indexes[idx])
      image = data['image']
      exp = data['category']
      image = image.unsqueeze(0).cuda()
      target = model(image)
      ans = (torch.argmax(target).cpu().numpy()+1)
      if (ans == exp):
        correct +=1
  return (correct/99)

 

In [0]:
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    accuracy = torch.tensor(0)
    count = 0

    adjust_learning_rate(optimizer, curr_lr)
    curr_lr *= GAMMA
    
    for c in range(0,int(4900/BATCH_SIZE)):
        optimizer.zero_grad()
        images = torch.zeros([4,3,224,224])
        targets = []
        for p in range(0,BATCH_SIZE):
            data = getitem(train_indexes[BATCH_SIZE*c+p])
            images[p,:,:,:] = data['image']
            targets.append(np.asscalar(data["category"])-1)
        targets = torch.LongTensor(targets)
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        
        outs = model(images)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()
                
        total_loss += loss.data
        accuracy += torch.sum(torch.argmax(outs, dim=1).cpu() == targets)
        count += BATCH_SIZE

        if c%200 == 0:
            print("Epoch:", epoch, "Iter:", c, "average batch loss:", total_loss/(c+1), "Accuracy:", accuracy.cpu().numpy()/count)
    
    print("\nEpoch:", epoch, "Total loss:", total_loss)
    acc = accuracy.cpu().numpy()/count
    print("Epoch:", epoch, "Accuracy:", acc,'\n') 
    
    model = model.eval()
    val_acc = validation_accuracy(model)
    print('Validation accuracy:',val_acc)
    validation_accuracies.append(val_acc)
    model = model.train()
    
    torch.save(model.state_dict(), "resnet152_epoch_"+str(epoch)+"pt")
    train_losses.append(total_loss/4900)
    train_accuracies.append(acc)

In [0]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.ylabel('loss')
plt.show()

In [0]:
import matplotlib.pyplot as plt
plt.plot(accuracies,'C0',label='Trainset accuracies')
plt.plot(validation_accuracies,'C1',label='Validation accuracies')
plt.ylabel('accuracy')
plt.show()

# Save/load model

In [0]:
torch.save(model.state_dict(), "resnet152_model0.pt")

In [0]:
model.load_state_dict(torch.load("resnet152_model0.pt"))
model.eval()

# Exporting solution to csv

In [0]:
from tqdm import tqdm

In [0]:
def answer_builder(model):
  """
  Performs inference on testset and exports predictions to answer.csv
  """
  model.eval()
  df = pd.DataFrame(columns=['id','category'])
  images =torch.empty([40000,3,224,224])
  with torch.no_grad():
    for idx in tqdm(range(0,40000)):
      img_name = os.path.join("/content/testing/", str(idx+1) + ".png")
      image = Image.open(img_name)
      image = tfms(image)
      image = image.unsqueeze(0)
      images[idx,:,:,:] = image
    print('\nPerforming inference...')
    images = images.cuda()
    target = model(images)
    target = torch.argmax(target, dim=1)
    print('Generating answer.csv')
    for i in tqdm(range(1,40001)):
      df.loc[i] = [i, torch.argmax(target).cpu().numpy()+1]

    df.to_csv('answer.csv', index=False)

In [0]:
answer_builder(model)